# TCP Server
Source - python Docs [socketserver](https://docs.python.org/3/library/socketserver.html)

In [ ]:
import socketserver
import time, random, threading
import numpy as np
signal_file, beat_file = './data/ecg/signal.npy', './data/ecg/beats.npy'
class ecg_gen:
    def __init__(self, sf, bf, sep=','):
        
        self.fs = 128 #Hz
        self.sf, self.bf = sf, bf
        self.sep=sep
        self.signal = np.load(sf)
        beats_data = np.load(bf)
        self.beats=beats_data[:,0].astype('int')
        self.labels = beats_data[:,1]
        self.nb = len(self.beats)
        self.reset()
    
    def reset(self):
        self.i = 0
        
    def tocsv(self,npA, sep=','):
        res=""
        for i in range(len(npA)):
            res+=str(npA[i])+sep
        return res
    
    def prepare_bytes(self):
        sep=self.sep
        self.i+=1
        # timestep, label, peakloc, signal_len, signal
        sig_slice = self.signal[self.beats[self.i-1]:self.beats[self.i+1]]
        res = str(self.i) + sep + \
              str(self.labels[self.i]) + sep + \
              str(self.beats[self.i]-self.beats[self.i-1]) + sep + \
              str(len(sig_slice)) + sep + \
              self.tocsv(sig_slice , sep=sep)

        return bytes(res, encoding='utf-8'), (self.i==self.nb-1)



# start server

In [ ]:
ecg = ecg_gen(signal_file, beat_file )

#<---- as found on python Docs socketserver
HOST, PORT = "localhost", 9999
class SERVER(socketserver.BaseRequestHandler):       
    def handle(self):
        """
        In addition to the literal forms, bytes objects can be created in a number of other ways:
            A zero-filled bytes object of a specified length: bytes(10)
            From an iterable of integers: bytes(range(20))
            Copying existing binary data via the buffer protocol: bytes(obj)
        """
        global ecg
        data_bytes, done = ecg.prepare_bytes()# --> Note: finally data is converted to bytes
        print('Sending:', len(data_bytes)/1024, ' MB')
        self.request.sendall(data_bytes)
        time.sleep(3) #<------------------ sleep time b/w handle
        if done:
            raise StopIteration
        

with socketserver.TCPServer((HOST, PORT), SERVER) as server:
    print('Started!')
    server.serve_forever() # interrupt the program with Ctrl-C
    
        

Started!
Sending: 5.2265625  MB
Sending: 5.173828125  MB
Sending: 5.0625  MB
Sending: 5.1796875  MB
Sending: 5.2431640625  MB
Sending: 5.25390625  MB
Sending: 5.349609375  MB
Sending: 5.435546875  MB
Sending: 5.34375  MB
Sending: 5.34375  MB
Sending: 5.326171875  MB
Sending: 5.232421875  MB
Sending: 5.3251953125  MB
Sending: 5.3115234375  MB
Sending: 5.2177734375  MB
Sending: 5.30859375  MB
Sending: 5.3544921875  MB
Sending: 5.3271484375  MB
Sending: 5.318359375  MB
Sending: 5.240234375  MB
Sending: 5.1591796875  MB
Sending: 5.1376953125  MB
Sending: 5.134765625  MB
Sending: 5.173828125  MB
Sending: 5.1474609375  MB
Sending: 5.0751953125  MB
Sending: 5.171875  MB
Sending: 5.224609375  MB
Sending: 5.130859375  MB
Sending: 5.2109375  MB
Sending: 5.26171875  MB
Sending: 5.1923828125  MB
Sending: 5.255859375  MB
Sending: 5.291015625  MB
Sending: 5.306640625  MB
Sending: 5.4072265625  MB
Sending: 5.4453125  MB
Sending: 5.478515625  MB
